### Métodos Numéricos II (2023.1 - T02)
- Aluno: Dario Filipe da Silva Costa
- Matrícula: 422156
- Email: dariocosta@alu.ufc.br

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython.display import display, Latex 

# Tarefa anterior

## Tarefa 10: Potência Regular
----
Ao pré multiplica um chute inicial de autovetor muitas vezes pela A o maior autovalor destaca o seu autovetor, e é possível extrair o autovalor com manipulações usando a A e o autovetor obtido, mas para funcionar é necessario que:

>$$\lambda(A)=\{\lambda_1, \lambda_2, \cdots, \lambda_n\}\text{ seja tal que }|\lambda_1|>|\lambda_2|\geq|\lambda_3| \geq \cdots \geq |\lambda_n|$$
> Um autovalor precisa ser maior que os demais (chamado de $\lambda_1$), e portanto também diferente de zero.

In [2]:
def potReg(A:np.ndarray, v0:np.ndarray, e:float=1e-7,
           iterMax=1e5, verboseGraph=False) -> tuple[np.ndarray, float]:
  '''
  Supõe v0 um vetor coluna e A uma matriz.

  PARAMETROS:
  - - - - -
  A  : ndarray - Matriz do problema
  v0 : ndarray - Chute inicial do autovetor x_1 (vetor coluna)
  '''
  v=v0.copy()
  lamb = 0

  iter = 0 #Contador de iterações
  if verboseGraph:
    hist={'iter':[], 'lambda_1':[]}
    
  while iter<=iterMax:
    v_old=v.copy()
    lamb_old = lamb
    v_old=(v_old)/((v_old.T@v_old)**0.5)
    v = A@v_old
    lamb = v_old.T@v

    iter+=1
    if verboseGraph:
      hist['iter'].append(iter)
      hist['lambda_1'].append(lamb)
    
    if abs((lamb - lamb_old)/lamb) < e: #Teste de parada
      break
  
  # Trecho apenas para caso o gráfico seja pedido e caso não tenha convergido.
  if verboseGraph:
    plt.plot(hist['iter'], hist['lambda_1'],'o--', label=f'$\\{list(hist.keys())[1]}$')
    plt.title('Estimativa do autovalor dominante')
    plt.grid(); plt.legend(); plt.show()    
  
  if iter > iterMax:
    print("As iterações superaram o máximo estipulado, chance de não ter convergido.")

  return v, lamb

# Tarefa 11: Potência Inversa e Deslocada

- Para convergencia da Potência inversa é necessário que:

>$$\lambda(A)=\{\lambda_1, \lambda_2, \cdots, \lambda_n\}\text{ seja tal que }|\lambda_1| \geq |\lambda_2|\geq|\lambda_3| \geq \cdots > |\lambda_n| > 0$$
> Um autovalor precisa ser menor que todos os demais e diferente de 0 (chamado de $\lambda_n$), e portanto todos são diferente de zero (a matriz tem inversa).
Com isso, na $A^{-1}$:
> $$\lambda(A^{-1})=\left\{\frac{1}{\lambda_n} = \bar{\lambda}_1, \frac{1}{\lambda_{n-1}}=\bar{\lambda}_2, \cdots, \frac{1}{\lambda_1}=\bar{\lambda}_n\right\}\\\\ \text{ é tal que }\\\\ |\bar{\lambda}_1| > |\bar{\lambda}_2| \geq |\bar{\lambda}_3| \geq \cdots \geq |\bar{\lambda}_n| > 0$$

In [3]:
def decompLU(A: np.ndarray)-> tuple[np.ndarray, np.ndarray]:
  '''
  Recebe A e devolve a decomposição em LU=A
  L= Lower Triag. e U = Upper Triag.
  '''
  n = len(A)
  L = np.zeros((n, n))
  U = np.zeros((n, n))
  for c in range(n):
    for l in range(c+1):
      sr = sum(L[l][k]*U[k][c] for k in range(l)) if l>0 else 0
      U[l][c] = A[l][c] - sr

    for l in range(c, n):
      sr=sum(L[l][k]*U[k][c] for k in range(c)) if c>0 else 0
      L[l][c] = (A[l][c]-sr)/U[c][c]

  return L, U

def solverLU(L: np.ndarray, U:np.ndarray, b:np.ndarray)->np.ndarray:
  ''' Devolve x que é solução do sistema LUx=b
  '''
  n = len(L)
  x = np.zeros(n)
  y = np.zeros(n)
  for i in range(n):
    r = sum(L[i][k]*y[k] for k in range(i)) if i>0 else 0
    y[i] = (b[i]-r)/L[i][i]
  for i in range(n-1, -1, -1):
    r = sum(U[i][k]*x[k] for k in range(i+1, n)) if i<n-1 else 0
    x[i] = (y[i]-r)/U[i][i]
  return x

OBS: Na biblioteca numpy já tem o solveLU implementado, mas por clareza e aprendizado implementei a minha.

In [4]:
def potInv(A:np.ndarray, v0:np.ndarray, e:float=1e-7,
           iterMax=1e5, verboseGraph=False) -> tuple[np.ndarray, float]:
  '''
  Supõe v0 um vetor coluna e A uma matriz.

  PARAMETROS:
  - - - - -
  A  : ndarray - Matriz do problema
  v0 : ndarray - Chute inicial do autovetor x_n (vetor coluna)
  '''
  v=v0.copy()
  lamb = 0
  # Obtendo a L e U <----
  L, U = decompLU(A)

  iter = 0 #Contador de iterações
  if verboseGraph:
    hist={'iter':[], 'lambda_n':[]}
    
  while iter<=iterMax:
    v_old=v.copy()
    lamb_old = lamb
    v_old=(v_old)/((v_old.T@v_old)**0.5)
    v = solverLU(L, U, v_old) #<-- Obtendo A^-1v_old=v_new pela
                              # solucao do sistema v_old=Av_new
    lamb = v_old.T@v

    iter+=1
    if verboseGraph:
      hist['iter'].append(iter)
      hist['lambda_n'].append(1/lamb)
    
    if abs((lamb - lamb_old)/lamb) < e: #Teste de parada
      break
  
  # Trecho apenas para caso o gráfico seja pedido e caso não tenha convergido.
  if verboseGraph:
    plt.plot(hist['iter'], hist['lambda_n'],'o--', label=f'$\\{list(hist.keys())[1]}$')
    plt.title('Estimativa do menor autovalor')
    plt.grid(); plt.legend(); plt.show()    
  
  if iter > iterMax:
    print("As iterações superaram o máximo estipulado, chance de não ter convergido.")

  return v, 1/lamb

In [5]:
def potDesloc(A:np.ndarray, v0:np.ndarray, u:float, e=1e-7)->tuple[np.ndarray, float]:
  I=np.identity(len(A))
  Abar = A-u*I
  v, lamb = potInv(Abar, v0, e)
  return v, lamb+u

###Testando para as matrizes:

- $A_1 = \left[\begin{array}{c c c}
5 & 2 & 1\\
2 & 3 & 1\\
1 & 1 & 2
\end{array}\right]$

- $A_2 = \left[\begin{array}{r r r}
-14 & 1 & -2\\
1 & -1 & 1\\
-2 & 1 & -11
\end{array}\right]$

- $A_3 = \left[\begin{array}{r r r r r}
40 & 8 & 4 & 2 & 1 \\
8 & 30 & 12 & 6 & 2\\
4 & 12 & 20 & 1 & 2\\
2 & 6 & 1 & 25 & 4\\
1 & 2 & 2 & 4 & 5
\end{array}\right]$

In [6]:
rnd = lambda x: round(x, 5)
As = {1:np.array([
       [5, 2, 1],
       [2, 3, 1], 
       [1, 1, 2]
       ]),
     2: np.array([
       [-14, 1, -2],
       [1, -1,   1], 
       [-2, 1, -11]
       ]),
     3:np.array([
       [40, 8, 4, 2, 1],
       [8, 30, 12, 6, 2],
       [4, 12, 20, 1, 2],
       [2, 6, 1, 25, 4],
       [1, 2, 2, 4, 5]
      ])
     }

In [7]:
def tryAll(A:np.ndarray, e=1e-7, iterMax=1e2)->dict:
  n=len(A)
  resp={}
  v = np.ones(n)

  x1, lamb1=potReg(A, v, e)
  xn, lambn=potInv(A, v, e)

  resp[lambn]=[xn, lambn]
  resp[lamb1]=[x1, lamb1]
  
  k=2
  iter = 2 # comeca em 2 pois vou usar no numero de divisões do intervalo
  while iter<iterMax and len(resp)<n:
    ks = list(sorted(resp.keys()))

    for i in range(len(ks)-1):
      #dividir o intervalo entre os autovalores já encontrados em iter partes
      us=[ks[i]+((ks[i+1]-ks[i])/iter)*k for k in range(1, iter)]

      for j in range(len(us)):
        xnew, lambnew = potDesloc(A, v, us[j], e)
      
        new = True
        for test in resp.keys():
          if abs(lambnew-test) < e: #Testando se ele já esta na lista
            new = False
        if new:
          resp[lambnew]=[xnew, lambnew] #adiconando o novo encontrado

  keys = list(sorted(resp.keys(), key=abs, reverse=True)) #organizando
  r = {k: resp[k]  for k in keys}
  return r


In [8]:
for A in As.values():
  r = tryAll(A)
  i = 1
  print(tabulate(A, tablefmt='grid'))
  l='$'
  for x, lamb in r.values():
    x=list(map(rnd, x))
    lamb=rnd(lamb)
    l += f'\lambda_{i}={lamb}, ~~'
    display(Latex(f'x_{i}={x}'))
    i+=1
  l+='$'
  display(Latex(l))

+---+---+---+
| 5 | 2 | 1 |
+---+---+---+
| 2 | 3 | 1 |
+---+---+---+
| 1 | 1 | 2 |
+---+---+---+


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>